# Task 1. Default Delay-Match-To-Sample (DMTS):

[This notebook implements task 1 as well, also using MNIST images. However this time the

A) Encoding of tasks

b) Model architecure

are different]

A sample image is presented and then, after a time delay of arbitrary duration, a test image is presented. The
goal is to evaluate the model's ability to retain the sample image in working memory during the delay period
and compare it to the test image. To titrate difficulty, the similarity between sample and test images can be
varied, and the model's invariance to image transformations and zero-shot generalization ability will also be
evaluated.


This notebook is structured as follows : 
1. Load task 1 and visualize it
2. Train (and evaluate/validate) a new architecture


[The two key differences with previous approaches are : 

a. Every time step (every image) has an associated label

b. The model produces one output per time step (per image)

]

In [151]:
import os
import numpy as np
from array import array
import matplotlib.pyplot as plt
import importlib

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms
from sklearn.metrics import accuracy_score

# Custom functions and modules
from auxFuns.data.task_loader import *
from auxFuns.data.task_visualizer import *

In [ ]:
# Reload the packages
import auxFuns.data.task_loader
importlib.reload(auxFuns.data.task_loader)

import auxFuns.data.task_visualizer
importlib.reload(auxFuns.data.task_visualizer)

In [ ]:
# import os
# os.chdir('..')
os.getcwd()

## 1. Load (and visualize) task 1

## 2. Train the newly created architecture (ConvNet embedder > Working Memory model > Linear classifier per time step)